In [0]:
pip install chart_studio

In [0]:
import pandas as pd
import math
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange, Legend, HoverTool, Label
from bokeh.transform import dodge
import bokeh.palettes
output_notebook()
# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# load all data
link_feb = 'https://drive.google.com/open?id=1vZDH_QZmct3Z1BbYdIKrq3ZqhvSV4xAh'
link_mar_apr = 'https://drive.google.com/open?id=16prdgB8ud13myHPlczGZpaa-BpUPpf35'

fluff_feb, id_feb = link_feb.split('=')
fluff_mar_apr, id_mar_apr = link_mar_apr.split('=')

downloaded = drive.CreateFile({'id':id_feb}) 
downloaded.GetContentFile('Fremkommelighedsanalyse_med_fraktil_og_potentialeHast_Feb.csv')  
df_feb = pd.read_csv('Fremkommelighedsanalyse_med_fraktil_og_potentialeHast_Feb.csv',sep=";",
                 header= 2,index_col=False,decimal=",")

downloaded = drive.CreateFile({'id':id_mar_apr}) 
downloaded.GetContentFile('Fremkommelighedsanalyse_med_fraktil_og_potentiale_Hast_MarApr.csv')  
df_mar_apr = pd.read_csv('Fremkommelighedsanalyse_med_fraktil_og_potentiale_Hast_MarApr.csv',sep=";",
                 header= 2,index_col=False,decimal=",")

Link_age_distribution= "https://drive.google.com/open?id=1kJlqevB6ORkT-p1bsdvGWETg0Av10h9X"
fluff_age_distribution, id_age_distribution = Link_age_distribution.split('=')


downloaded = drive.CreateFile({'id':id_age_distribution}) 
downloaded.GetContentFile('Data_aldersfordeling.xlsx')  
df_age_distribution = pd.read_excel('Data_aldersfordeling.xlsx')

Link_unemployment= "https://drive.google.com/open?id=17oBxPfqj-yiHulSP1-WwbtXJZDrURxiI"
fluff_unemployment, id_unemployment = Link_unemployment.split('=')


downloaded = drive.CreateFile({'id':id_unemployment}) 
downloaded.GetContentFile('Data_fuldtidsledige.xlsx')  
df_unemployment = pd.read_excel('Data_fuldtidsledige.xlsx')


Link_income= "https://drive.google.com/open?id=1J-xbmlKRbNkbI_S4egod3RHc2CtpiWhD"
fluff_income, id_income = Link_income.split('=')


downloaded = drive.CreateFile({'id':id_income}) 
downloaded.GetContentFile('Income2.xlsx')  
df_income = pd.read_excel('Income2.xlsx')

In [0]:
# Define the municipality of interest 
focus_municipality = ['København','Frederiksberg', 'Ballerup', 'Brøndby', 'Dragør', 'Gentofte', 'Gladsaxe', 'Glostrup','Herlev',
                      'Albertslund','Hvidovre', 'Høje-Taastrup', 'Rødovre', 'Ishøj','Tårnby', 'Vallensbæk']

# extract the data for average speed for the covid periode
cols = ['StopPointSectionMunicipalityDisplayName','Hastighed_observeret__netto__km_t_','Hastighed_observeret__brutto__km_t_']
df_mar_apr_avg_hast = df_mar_apr[df_mar_apr['StopPointSectionMunicipalityDisplayName'].isin(focus_municipality)][cols]

df_mar_apr_avg_hast = df_mar_apr_avg_hast.groupby('StopPointSectionMunicipalityDisplayName', as_index=False).agg(['sum', 'count'])

df_mar_apr_avg_hast['avg_speed_netto'] = df_mar_apr_avg_hast['Hastighed_observeret__netto__km_t_']['sum'].values/df_mar_apr_avg_hast['Hastighed_observeret__netto__km_t_']['count'].values
df_mar_apr_avg_hast['avg_speed_brutto'] = df_mar_apr_avg_hast['Hastighed_observeret__brutto__km_t_']['sum'].values/df_mar_apr_avg_hast['Hastighed_observeret__brutto__km_t_']['count'].values

df_mar_apr_avg_hast = df_mar_apr_avg_hast[['avg_speed_netto','avg_speed_brutto']]

# extract the data for average speed febuar (prior to covid)
df_feb_avg_hast = df_feb[df_feb['StopPointSectionMunicipalityDisplayName'].isin(focus_municipality)][cols]

df_feb_avg_hast = df_feb_avg_hast.groupby('StopPointSectionMunicipalityDisplayName', as_index=False).agg(['sum', 'count'])

df_feb_avg_hast['avg_speed_netto'] = df_feb_avg_hast['Hastighed_observeret__netto__km_t_']['sum'].values/df_feb_avg_hast['Hastighed_observeret__netto__km_t_']['count'].values
df_feb_avg_hast['avg_speed_brutto'] = df_feb_avg_hast['Hastighed_observeret__brutto__km_t_']['sum'].values/df_feb_avg_hast['Hastighed_observeret__brutto__km_t_']['count'].values

df_feb_avg_hast = df_feb_avg_hast[['avg_speed_netto','avg_speed_brutto']]

# extract average length of stopping time for covid periode
cols = ['StopPointSectionMunicipalityDisplayName','Opholdstid_observeret_ved_stop']
df_mar_apr_stop_t = df_mar_apr[df_mar_apr['StopPointSectionMunicipalityDisplayName'].isin(focus_municipality)][cols]

df_mar_apr_stop_t = df_mar_apr_stop_t.groupby('StopPointSectionMunicipalityDisplayName', as_index=False).agg(['sum', 'count'])

df_mar_apr_stop_t['avg_stop_t'] = df_mar_apr_stop_t['Opholdstid_observeret_ved_stop']['sum'].values/df_mar_apr_stop_t['Opholdstid_observeret_ved_stop']['count'].values

# extract average length of stopping time for febuar (prior to covid)
df_feb_stop_t = df_feb[df_feb['StopPointSectionMunicipalityDisplayName'].isin(focus_municipality)][cols]

df_feb_stop_t = df_feb_stop_t.groupby('StopPointSectionMunicipalityDisplayName', as_index=False).agg(['sum', 'count'])

df_feb_stop_t['avg_stop_t'] = df_feb_stop_t['Opholdstid_observeret_ved_stop']['sum'].values/df_feb_stop_t['Opholdstid_observeret_ved_stop']['count'].values

# joining avarage speed for febuar and covid periode
df_feb_avg_hast.columns = ['avg_speed_netto_feb', 'avg_speed_brutto_feb']
df_mar_apr_avg_hast.columns = ['avg_speed_netto_mar_apr', 'avg_speed_brutto_mar_apr']
df_avg_hast = df_feb_avg_hast.merge(df_mar_apr_avg_hast, how = 'inner',left_index = True, right_index = True)
df_avg_hast = df_avg_hast[['avg_speed_brutto_feb','avg_speed_brutto_mar_apr']]
df_avg_hast.reset_index(inplace=True)

def ageInt(x):
    return (int(x.split()[0]))

df_age_distribution['Age'] = df_age_distribution['Age'].apply(ageInt)

df_age_distribution = df_age_distribution[df_age_distribution['Municipality'].isin(focus_municipality)]
df_age_distribution = df_age_distribution.sort_values(['Municipality','Age'])
focus_municipality.sort()

median_ages = []
for m in focus_municipality:
  df_tmp = df_age_distribution[df_age_distribution['Municipality']== m]
  median_num = np.round(sum(df_tmp['Number'].values)/2)
  count = 0
  for a in np.unique(df_tmp['Age']):
    count += df_tmp.iloc[a]['Number']
    if count >= median_num:
      median_ages.append(a)
      break

data = {'Municipality': focus_municipality,
        'Midian age': median_ages
        }

df_midian_age = pd.DataFrame(data, columns = ['Municipality', 'Midian age'])

In [0]:
##################################################################################################################################################
###################################################################################################################################################
####################### Barplot with avg. onboarding per day for each municipality with before and after lockdown #################################
###################################################################################################################################################
###################################################################################################################################################

#Creating format fitting for bokeh
source = ColumnDataSource(df_avg_hast)

#Creating a list for the x-axis on the bar-chart. 
x_range = [str(m) for m in source.data['StopPointSectionMunicipalityDisplayName']]

#Generating 14 colors for the different focus crimes  
colors = bokeh.palettes.Category20[20]

#Creating figure frame with title, x -and y labels and disabling toolbar. 
p = figure(x_range=x_range,
           title='Histogram over avarage speed of busses in the Municipality',
           x_axis_label='Municipality',
           y_axis_label='average speed km / h',
           plot_width=950,plot_height=500,
           toolbar_location=None)

bar ={} # to store vbars
for indx,i in enumerate(source.column_names[-2:]):
    bar[i] = p.vbar(x='StopPointSectionMunicipalityDisplayName', top=i, source= source, width=0.8, 
                 fill_alpha=0.5,line_color=None,
                 color=colors[indx*5],visible=False) 
    
items = [(i,[bar[i]]) for i in source.column_names[-2:]] #Creating list of tuples with focus crime name and belonging bar chart. 
legend = Legend(items=items,location=(0,90)) # Creating legends with 'items'
p.add_layout(legend,'left') # Adding legends to ´p´ and setting location.

p.legend.click_policy="hide"
p.legend.title = 'To see a periode, choose one below'

show(p)

In [0]:
source.column_names

['index',
 'StopPointSectionMunicipalityDisplayName',
 'avg_speed_brutto_feb',
 'avg_speed_brutto_mar_apr']

In [0]:
len(colors)

6

In [0]:
#################################################################################################################################################
#################################################################################################################################################
#################### Jitter plot to spot correlation between speed increase and distance from center of Copenhagen. #############################
#################################################################################################################################################
#################################################################################################################################################

####################################################### set up data frame for the plot ######################################################
# coordinates for city halls
coor_copehagen = [55.675579, 12.570196]
coor_gentofte = [55.749205, 12.557447]
coor_Dragør = [55.596778, 12.637316]
coor_Frederiksberg = [55.678651, 12.532559]
coor_LyngbyTaarbæk = [55.772372, 12.504714]
coor_Tårnby = [55.634091, 12.619329]
coor_Gladsaxe = [55.743183, 12.497172]
coor_Herlev = [55.725144, 12.432609]
coor_Ballerup = [55.734340, 12.364389]
coor_Glostrup = [55.665082, 12.394101]
coor_Hvidovre = [55.643127, 12.477458]
coor_HøjeTaastrup = [55.652252, 12.264783]
coor_Brøndby = [55.652989, 12.419191]
coor_Albertslund = [55.657088, 12.357865]
coor_Ishøj = [55.614225, 12.355672]

# create a dict with coordinates
coor_town_halls = {'Albertslund':coor_Albertslund,
                  'Ballerup':coor_Ballerup,
                  'Brøndby': coor_Brøndby,
                  'Dragør': coor_Dragør,
                  'Frederiksberg': coor_Frederiksberg,
                  'Gentofte': coor_gentofte,
                  'Gladsaxe': coor_Gladsaxe,
                  'Glostrup': coor_Glostrup,
                  'Herlev': coor_Herlev,
                  'Hvidovre': coor_Hvidovre,
                  'Høje-Taastrup': coor_HøjeTaastrup,
                  'Ishøj': coor_Ishøj,
                  'København': coor_copehagen,
                  'Lyngby-Taarbæk': coor_LyngbyTaarbæk,
                  'Tårnby':coor_Tårnby}

# sort df and list of municipality to make the addition of the new col easy
df_hast_dist = df_avg_hast.sort_values('StopPointSectionMunicipalityDisplayName')
focus_municipality.sort()

# calculate the distance between copenhagen city hall and the other city halls
# list to store distances
distances = []

# radium of the earth
R = 6373.0

# coordinates for copenhagen city hall 
lat_center, lon_center = coor_town_halls['København']

# convert to radians:
lat_center = math.radians(lat_center)
lon_center = math.radians(lon_center)

for m in focus_municipality:
  # get coordinates for curent Municipality
  lat, lon = coor_town_halls[m]
  
  # convert to radians
  lat = math.radians(lat)
  lon = math.radians(lon)

  # difference i the coordinates
  dlon = lon - lon_center
  dlat = lat - lat_center

  a = math.sin(dlat / 2)**2 + math.cos(lat_center) *math.cos(lat) * math.sin(dlon / 2)**2

  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

  d = R*c

  distances.append(d)

# prepare the data frame
df_hast_dist['distance'] = distances
df_hast_dist['Speed_change'] = df_hast_dist['avg_speed_brutto_mar_apr'].values - df_hast_dist['avg_speed_brutto_feb'].values 
df_hast_dist = df_hast_dist[['StopPointSectionMunicipalityDisplayName','Speed_change','distance']]
############################################# regression #####################################################################################


############################################ create the plot ##################################################################################
# define source
source = ColumnDataSource(data = df_hast_dist)
# define tooltip
TOOLTIPS = [("Municipality: ", "@StopPointSectionMunicipalityDisplayName"),
            ("Distance", "@distance"),
            ("Speed change","@Speed_change")]
# create figure
f = figure(title = 'Distances to city center vs change in speed',
           x_axis_label='Distance in km',
           y_axis_label='change km/h',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)
# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

#Create linear regression line
cor = df_hast_dist[['Speed_change','distance']].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)


R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)
f.line(x="Speed_change", y = "predicted_decrease",source=source,color="red")

# Create plot
f.circle(x = 'distance', y = 'Speed_change', size=10,source=source)
# show plot
show(f)

KeyError: ignored

In [0]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Median age vs. avg. increase in speed ##############################################
#################################################################################################################################################
#################################################################################################################################################
from sklearn.linear_model import LinearRegression
############ prepare data ##############################################################################################
speed_change = df_avg_hast['avg_speed_brutto_mar_apr'].values - df_avg_hast['avg_speed_brutto_feb'].values 
df_median_age_avg_speed = df_midian_age.copy()
df_median_age_avg_speed['Speed_change'] = speed_change
df_median_age_avg_speed.columns = ['Municipality','Midian_age','Speed_change']


X = df_median_age_avg_speed.Midian_age.to_numpy().reshape(-1,1)
Y = df_median_age_avg_speed.Speed_change.to_numpy().reshape(-1,1)

linear_regressor = LinearRegression()

linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X)

Y_pred = list(Y_pred.squeeze())

df_median_age_avg_speed["predicted_decrease"] = Y_pred


cor = df_median_age_avg_speed[['Midian_age','Speed_change']].corr()

R_sqrt = round(cor.iloc[0,1]**2,3)
########### plot #########################################################################################################
source = ColumnDataSource(data = df_median_age_avg_speed)

TOOLTIPS = [("Municipality: ", "@Municipality"),
            ("Median age", "@Midian_age"),
            ("Speed change","@Speed_change")]
# create figure
f = figure(title = 'Median age vs change in avg. speed',
           x_axis_label='Median age',
           y_axis_label='change km/h',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)
# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

R_sqrt_Label = Label(x=0, y=290, x_units='screen', y_units='screen',
                 text=f'  R^2 value: {R_sqrt}  ', render_mode='canvas',
                 border_line_color='black', border_line_alpha=1.0,
                 background_fill_color='white', background_fill_alpha=1.0,
                 text_font_size='14pt',text_font="times")
f.add_layout(R_sqrt_Label)
f.line(x="Speed_change", y = "predicted_decrease",source=source,color="red")



# Create plot
f.circle(x = 'Midian_age', y = 'Speed_change', size=10,source=source)
# show plot
show(f)

In [0]:
df_income.head()

,Municipality,25-29 år,30-34 år,35-39 år,Avg
0,København,177571,241580,276706,231952.3333
1,Frederiksberg,196081,285249,331237,270855.6667
2,Dragør,179750,293054,364511,279105.0000
3,Tårnby,189619,255956,295934,247169.6667
4,Albertslund,165023,218541,246504,210022.6667


In [0]:
def bar_plot(x_range, title, x_lab, y_lab,x,y):
  f = figure(x_range = x_range,
           title = title,
           x_axis_label=x_lab,
           y_axis_label = y_lab ,      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, toolbar_location=None, tools="")
  # center title
  f.title.align = 'center'
  # set title font size
  f.title.text_font_size = '20px'
  # set title font type
  f.title.text_font = 'times'
  # set axes labels font size
  f.axis.axis_label_text_font_size = '16px'
  # set axes labels font type
  f.axis.axis_label_text_font = 'times'
  # rotate x values
  f.xaxis.major_label_orientation = 1.2
  f.vbar(x= x, top=y, width=0.9)

  f.xgrid.grid_line_color = None
  f.y_range.start = 0

  return(f)


In [0]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Muni vs. income: ########################################################
#################################################################################################################################################
#################################################################################################################################################

f = bar_plot(x_range=df_income['Municipality'].values,title= 'Average income in municipality',
             x_lab= 'Municipality', y_lab = 'Average disponoble income kr', x = df_income['Municipality'].values,
             y = df_income['Avg'].values)

show(f)

In [0]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Transport vs. unemployment: ##############################################
#################################################################################################################################################
#################################################################################################################################################

############ prepare data ##############################################################################################


df_age_distribution_working = df_age_distribution[(df_age_distribution['Age'] > 24) & (df_age_distribution['Age'] < 68)]

df_unemployment = df_unemployment[df_unemployment['Municipality'].isin(focus_municipality)]

df_age_distribution_working_unemployment  = pd.merge(df_age_distribution_working ,df_unemployment,how='inner')

df_age_distribution_working_unemployment = df_age_distribution_working_unemployment.groupby('Municipality').sum()

df_age_distribution_working_unemployment['procent'] = df_age_distribution_working_unemployment['2020M02']/43/df_age_distribution_working_unemployment['Number']

##################################### Make plot ##########################################################################################
f = bar_plot(x_range=list(df_age_distribution_working_unemployment.index),title= 'Unemployment % in municipality',
             x_lab= 'Municipality', y_lab = 'Unemployment %', x = list(df_age_distribution_working_unemployment.index),
             y = df_age_distribution_working_unemployment['procent'].values)

show(f)

In [0]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Median Age in Municipality ####################################################
#################################################################################################################################################
#################################################################################################################################################
f = bar_plot(x_range=df_midian_age['Municipality'].values,title= "Median Age in Municipality",
             x_lab= 'Municipality', y_lab = 'Median Age', x = df_midian_age['Municipality'].values,
             y = df_midian_age['Midian age'].values)

show(f)

In [0]:
#################################################################################################################################################
#################################################################################################################################################
############################################################ Median age vs. avg. increase in speed ##############################################
#################################################################################################################################################
#################################################################################################################################################

############ prepare data ##############################################################################################
df_unemployment_speed_change = df_age_distribution_working_unemployment.merge(df_median_age_avg_speed, how = 'inner', left_index =True, right_index = False, on = 'Municipality')[['Municipality','procent','Speed_change']]
########### plot #########################################################################################################
source = ColumnDataSource(data = df_unemployment_speed_change)

TOOLTIPS = [("Municipality: ", "@Municipality"),
            ("Median age", "@procent"),
            ("Speed change","@Speed_change")]
# create figure
f = figure(title = 'unemployment % vs change in avg. speed',
           x_axis_label='unemployment %',
           y_axis_label='change km/h',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS)
# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = 'procent', y = 'Speed_change', size=10,source=source)
# show plot
show(f)

In [0]:
df_avg_hast['speed_change'] = (df_avg_hast['avg_speed_brutto_mar_apr']/df_avg_hast['avg_speed_brutto_feb'])-1
df_speed_income = df_avg_hast.merge(df_income, how='inner',left_on = 'StopPointSectionMunicipalityDisplayName', right_on='Kommune')

In [0]:
source = ColumnDataSource(data = df_speed_income)

TOOLTIPS = [("Municipality: ", "@Kommune"),
            ("Income: ", "@2018"),
            ("Speed change: ","@speed_change")]
# create figure
f = figure(title = 'Income vs change in avg. speed %',
           x_axis_label='Income kr.',
           y_axis_label='change in avg speed %',      
           background_fill_color = "beige",background_fill_alpha = 0.8, 
           border_fill_color = "beige", border_fill_alpha = 0.8, tooltips=TOOLTIPS,plot_width = 400, plot_height = 400)
# center title
f.title.align = 'center'
# set title font size
f.title.text_font_size = '20px'
# set title font type
f.title.text_font = 'times'
# set axes labels font size
f.axis.axis_label_text_font_size = '16px'
# set axes labels font type
f.axis.axis_label_text_font = 'times'

# Create plot
f.circle(x = '2018', y = 'speed_change', size=10,source=source)
# show plot
show(f)

In [0]:
df_speed_income

,StopPointSectionMunicipalityDisplayName,avg_speed_brutto_feb,avg_speed_brutto_mar_apr,speed_change,Kommune,2018
0,Albertslund,26.508728,28.439604,0.072839,Albertslund,208399
1,Ballerup,27.425342,29.179883,0.063975,Ballerup,236121
2,Brøndby,28.946531,30.674521,0.059696,Brøndby,211029
3,Dragør,32.189848,33.613643,0.044231,Dragør,311793
4,Frederiksberg,18.191472,19.368576,0.064706,Frederiksberg,276683
5,Gentofte,25.093804,26.670838,0.062846,Gentofte,427370
6,Gladsaxe,23.298687,24.457107,0.049720,Gladsaxe,250679
7,Glostrup,24.680453,26.228632,0.062729,Glostrup,232928
8,Herlev,22.115820,23.583667,0.066371,Herlev,236643
9,Hvidovre,25.136425,26.529289,0.055412,Hvidovre,229211
